In [1]:
from requests import Request, Session
import pandas as pd
import json
import time
import webbrowser
import pprint
import gspread
import csv
from datetime import date, timedelta
from dateutil import parser
import numpy as np

headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': 'insert_coinmarketcap_api_key_here'
}

session = Session()
session.headers.update(headers)

today = date.today()
d_min = date.today() - timedelta(days=365)

## Get Token List

In [2]:
url_map_id = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/map'
parameters_map_id = { 'start' : 1,
                      'sort' : 'cmc_rank',
                    }
response_map_id = session.get(url_map_id, params = parameters_map_id)
token_list = pd.json_normalize(json.loads(response_map_id.text)['data'])
token_list.sample()

,id,name,symbol,slug,rank,displayTV,is_active,first_historical_data,last_historical_data,platform,platform.id,platform.name,platform.symbol,platform.slug,platform.token_address
3315,16275,Duckie Land,MMETA,duckie-land,3316,1,1,2021-12-20T09:51:28.000Z,2022-10-31T07:36:00.000Z,NaN,1839.0,BNB,BNB,bnb,0x7a9c8d33963aecca9a821802adfaf5bd9392351f


## Get Token Historical Data OHLCV

In [8]:
url_ohlcv = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/ohlcv/historical'
slug_list = ['bitcoin','ethereum','tokenomy','pintu-token','tokocrypto']
df = pd.DataFrame()

for i in slug_list:
  parameters_ohlcv = { 'slug' : i,
                      'time_period' : 'daily',
                      'time_start' : d_min,
                      'time_end' : today,
                      'interval' : 'daily'
                    }
  response_ohlcv = session.get(url_ohlcv, params = parameters_ohlcv)
  ohlcv_history = pd.json_normalize(json.loads(response_ohlcv.text)['data']['quotes'])
  ohlcv_history['assets'] = i 
  df = pd.concat([df,ohlcv_history])

df['time_open'] = pd.to_datetime(df['time_open'])
df['time_close'] = pd.to_datetime(df['time_close'])
df['time_high'] = pd.to_datetime(df['time_high'])
df['time_low'] = pd.to_datetime(df['time_low'])
df.sample(5)

,time_open,time_close,time_high,time_low,quote.USD.open,quote.USD.high,quote.USD.low,quote.USD.close,quote.USD.volume,quote.USD.market_cap,quote.USD.timestamp,assets
36,2021-12-31 00:00:00+00:00,2021-12-31 23:59:59.999000+00:00,2021-12-31 12:03:00+00:00,2021-12-31 04:51:00+00:00,1.304998,1.323849,1.293549,1.300500,7.445192e+04,0.000000e+00,2021-12-31T23:59:59.999Z,pintu-token
109,2022-02-18 00:00:00+00:00,2022-02-18 23:59:59.999000+00:00,2022-02-18 12:29:00+00:00,2022-02-18 06:05:00+00:00,0.740587,0.840096,0.731920,0.775510,5.289747e+07,8.414279e+07,2022-02-18T23:59:59.999Z,tokocrypto
335,2022-10-26 00:00:00+00:00,2022-10-26 23:59:59.999000+00:00,2022-10-26 09:46:00+00:00,2022-10-26 10:41:00+00:00,0.596809,0.641324,0.586931,0.617683,1.740189e+05,1.443039e+07,2022-10-26T23:59:59.999Z,pintu-token
136,2022-03-17 00:00:00+00:00,2022-03-17 23:59:59.999000+00:00,2022-03-17 05:02:00+00:00,2022-03-17 07:47:00+00:00,41140.844816,41287.535204,40662.870601,40951.379876,2.200960e+10,7.775152e+11,2022-03-17T23:59:59.999Z,bitcoin
121,2022-03-02 00:00:00+00:00,2022-03-02 23:59:59.999000+00:00,2022-03-02 07:44:00+00:00,2022-03-02 13:23:00+00:00,0.045329,0.045933,0.044873,0.045109,1.408165e+04,9.021803e+06,2022-03-02T23:59:59.999Z,tokenomy


In [4]:
url_map_id = 'https://pro-api.coinmarketcap.com/v2/cryptocurrency/market-pairs/latest'
parameters_map_id = { 'slug' : 'tokocrypto',
                      'sort' : 'cmc_rank'
                    }
response_map_id = session.get(url_map_id, params = parameters_map_id)
market_pairs = pd.json_normalize(json.loads(response_map_id.text)['data']['market_pairs'])
market_pairs = market_pairs[['market_pair','category','exchange.name','quote.exchange_reported.volume_24h_quote']]
market_pairs.sample(5)

,market_pair,category,exchange.name,quote.exchange_reported.volume_24h_quote
3,TKO/USDT,spot,Bibox,106655.149447
15,TKO/USDT,spot,Gate.io,13705.073485
11,TKO/USDT,spot,Phemex,39603.776427
9,TKO/USDT,spot,Pionex,54756.342701
17,TKO/BTC,spot,Tokocrypto,0.633197


In [5]:
url_exc_id = 'https://pro-api.coinmarketcap.com/v1/exchange/map'
parameters_exc_id = { 'start' : 1,
                      'limit' : 5000
                    }
response_exc_id = session.get(url_exc_id, params = parameters_exc_id)
exc_pairs = pd.json_normalize(json.loads(response_exc_id.text)['data'])
#exc_pairs = exc_pairs[['market_pair','category','exchange.name','quote.exchange_reported.volume_24h_quote']]
exc_pairs.loc[exc_pairs['name'] == 'Reku']

,id,name,slug,is_active,first_historical_data,last_historical_data
175,723,Reku,rekeningku-com,1,2020-05-12T21:40:00.000Z,2022-10-31T07:40:00.000Z


In [6]:
url_quote_id = 'https://pro-api.coinmarketcap.com/v1/exchange/quotes/historical'
parameters_quote_id = { 'slug' : 'rekeningku-com',
                      'time_start' : d_min,
                      'time_end' : today,
                      'interval' : 'daily'
                    }
response_quote_id = session.get(url_quote_id, params = parameters_quote_id)
quote_hist = pd.json_normalize(json.loads(response_quote_id.text)['data']['quotes'])
#exc_pairs = exc_pairs[['market_pair','category','exchange.name','quote.exchange_reported.volume_24h_quote']]
quote_hist.sample()

,timestamp,num_market_pairs,quote.USD.volume_24h,quote.USD.timestamp
321,2022-09-18T00:00:12.000Z,41,7.377527e+06,2022-09-18T00:00:12.000Z


In [9]:
#market_pairs.to_csv('/Users/ghazian/Downloads/market_pairs.csv')
token_list.to_csv('/Users/ghazian/Downloads/token_list.csv')
df.to_csv('/Users/ghazian/Downloads/ohlcv.csv')